### SGD(Stochastic Gradient Descent) 이용한 행렬분해 
- R = P*Q -> Real R과 가깝도록(오류 최소화하여) 반복하여 P,Q 찾기 

#### Algorithm
- 1. P,Q 임의로 설정 
- 2. pred(R) = P*t(Q) -> Error= pred(R) - real(R) 
- 3. minimize(Error) -> update P,Q !!
- 4. 반복

- L2-norm Cost function은 아래와 같다.
   - min(sum(realR - P*t(Q))) + lambda(q^2 + p^2) 

In [11]:
import numpy as np

# R 생성, P,Q 초기화, K=3 으로 설정 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3,1],
              [np.NaN, np.NaN, 3,4,4],
              [5,2,1,2,np.NaN]])
num_users, num_items= R.shape
K=3 # 잠재요인의 차원 

print(num_users, num_items, K) # R은 4x4 행렬

np.random.seed(1)
P= np.random.normal(scale = 1./K, size = (num_users, K)) 
Q= np.random.normal(scale = 1./K, size = (num_items, K))
    # scale(표준편차) 디폴트 1 인데 1/K로 조정 
    # 행렬 size 지정(잠재요인 차원수를 col으로 맞춰줌)

print(P) 
print(Q)
print(Q.T) # transpose , size = (K, num_items) 

4 5 3
[[ 0.54144845 -0.2039188  -0.17605725]
 [-0.35765621  0.28846921 -0.76717957]
 [ 0.58160392 -0.25373563  0.10634637]
 [-0.08312346  0.48736931 -0.68671357]]
[[-0.1074724  -0.12801812  0.37792315]
 [-0.36663042 -0.05747607 -0.29261947]
 [ 0.01407125  0.19427174 -0.36687306]
 [ 0.38157457  0.30053024  0.16749811]
 [ 0.30028532 -0.22790929 -0.04096341]]
[[-0.1074724  -0.36663042  0.01407125  0.38157457  0.30028532]
 [-0.12801812 -0.05747607  0.19427174  0.30053024 -0.22790929]
 [ 0.37792315 -0.29261947 -0.36687306  0.16749811 -0.04096341]]


In [12]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 함수 만들기 

from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0 # 초기값 
    
    # pred R 생성하기 - P와 t(Q)의 내적으로 
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 Null값이 아닌 값의 인덱스 추출해서 실제 R 행렬과 예측 행렬의 RMSE 계산
    x_non_zero_ind= [non_zero[0] for non_zero in non_zeros] # non-zero index - x
    y_non_zero_ind= [non_zero[1] for non_zero in non_zeros] # non-zero-index - y
    
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros= full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    
    # 최종 rmse 계산
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse= np.sqrt(mse)
    
    return rmse

In [13]:
# R >0 행위치, 열위치, 값을 non-zeros 리스트에 저장 
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [14]:
### Parameter Setting 
steps=1000
learning_rate=0.01
r_lambda=0.01


# SGD로 P, Q matrix upate 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        
        # L2- norm 적용한 SGD update
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    
    if (step % 50) == 0 :# 중간확인
        print("### iteration step : ", step," rmse : ", rmse)


### iteration step :  0  rmse :  3.2388050277987723
### iteration step :  50  rmse :  0.4876723101369648
### iteration step :  100  rmse :  0.1564340384819247
### iteration step :  150  rmse :  0.07455141311978046
### iteration step :  200  rmse :  0.04325226798579314
### iteration step :  250  rmse :  0.029248328780878973
### iteration step :  300  rmse :  0.022621116143829466
### iteration step :  350  rmse :  0.019493636196525135
### iteration step :  400  rmse :  0.018022719092132704
### iteration step :  450  rmse :  0.01731968595344266
### iteration step :  500  rmse :  0.016973657887570753
### iteration step :  550  rmse :  0.016796804595895633
### iteration step :  600  rmse :  0.01670132290188466
### iteration step :  650  rmse :  0.01664473691247669
### iteration step :  700  rmse :  0.016605910068210026
### iteration step :  750  rmse :  0.016574200475705
### iteration step :  800  rmse :  0.01654431582921597
### iteration step :  850  rmse :  0.01651375177473524
### iterati

In [18]:
pred_matrix = np.dot(P, Q.T)
print("예측행렬 :\n", np.round(pred_matrix, 3))
print("실제행렬 :\n", R) 

예측행렬 :
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
실제행렬 :
 [[ 4. nan nan  2. nan]
 [nan  5. nan  3.  1.]
 [nan nan  3.  4.  4.]
 [ 5.  2.  1.  2. nan]]


- NULL값이 아닌 값은 비슷한 값으로 잘 나옴 확인.
- NULL값은 새로운 예측값으로 채워짐.